#### Lab 02 - Collaborative Filtering (User, Film, Rating)

Maintain User Engagement by Generating Recommendation Using Other User Preference

In [1]:
import warnings, pandas

warnings.filterwarnings("ignore")

In [2]:
rating = pandas.read_table("ratings.csv", sep=",")

rating.iloc[:5]

,UserID,MovieID,Rating,Timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
rating.tail()

,UserID,MovieID,Rating,Timestamp
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352
100835,610,170875,3.0,1493846415


In [4]:
# Check Null Value on Rating Table

null = rating.isnull().sum()

null.sum()

0

In [5]:
# Check Average Rating on Rating Table

rata_rata_rating = rating["Rating"].mean()

rata_rata_rating

3.501556983616962

In [6]:
# Get Total Film

# Our Total Film is 9.724 Film

total_film = rating["MovieID"].nunique()

total_film

9724

In [7]:
# Get Total User

# Our Total User is 610 User

total_user = rating["UserID"].nunique()

total_user

610

In [8]:
user_rating = rating.groupby("UserID")["Rating"].agg(["count", "mean"])

user_rating.columns = ["Tonton", "Rating"]

user_rating.sort_values("Rating", ascending=False).iloc[:5]

,Tonton,Rating
UserID,,
53,20,5.000000
251,23,4.869565
515,26,4.846154
25,26,4.807692
30,34,4.735294


In [9]:
user_rating = user_rating[user_rating["Tonton"] > 50]

user_rating.sort_values("Rating", ascending=False).iloc[:5]

,Tonton,Rating
UserID,,
523,75,4.693333
348,55,4.672727
171,82,4.634146
452,202,4.556931
43,114,4.552632


In [10]:
film_rating = rating.groupby("MovieID")["Rating"].agg(["count", "mean"])

film_rating.columns = ["Tonton", "Rating"]

film_rating.sort_values("Rating", ascending=False).iloc[:5]

,Tonton,Rating
MovieID,,
88448,1,5.0
100556,1,5.0
143031,1,5.0
143511,1,5.0
143559,1,5.0


In [11]:
film_rating = film_rating[film_rating["Tonton"] > 10]

film_rating.sort_values("Rating", ascending=False).iloc[:5]

,Tonton,Rating
MovieID,,
1041,11,4.590909
3451,11,4.545455
1178,12,4.541667
1104,20,4.475000
2360,12,4.458333


In [12]:
table_film = pandas.read_table("movies.csv", sep=",")

table_film["Genre"] = table_film["Genre"].apply(lambda i : " ".join(i.split("|")))

table_film = table_film[table_film["Genre"] != "(no genres listed)"]

replacer = {"Sci-Fi":"SciFi", "Film-Noir":"Filnoir"}

for i, t in replacer.items():

  table_film["Genre"] = table_film["Genre"].str.replace(i, t)

table_film.iloc[:5]

,MovieID,Title,Genre
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy


In [13]:
rating_table = rating.pivot_table(index="MovieID", columns="UserID", values="Rating")

SparseRate = lambda table : round((table.isnull().sum().sum() / table.size) * 100, 3)

SparseRate(rating_table)

98.3

In [14]:
minimum_film_interaction = 10

WatchFilm = rating.groupby("MovieID")["Rating"].agg(["count", "mean"])

WatchFilm.columns = ["TotalWatch", "Rating"]

WatchFilm = WatchFilm[WatchFilm["TotalWatch"] > minimum_film_interaction]

rating = rating[rating["MovieID"].isin(WatchFilm.index)]

rating["MovieID"].nunique()

2121

In [15]:
rating_table = rating.pivot_table(index="MovieID", columns="UserID", values="Rating")

SparseRate(rating_table)

93.845

In [16]:
minimum_user_interaction = 50

UserWatch = rating.groupby("UserID")[["Rating"]].count()

UserWatch = UserWatch[UserWatch["Rating"] > minimum_user_interaction]

rating = rating[rating["UserID"].isin(UserWatch.index)]

In [17]:
rating_table = rating.pivot_table(index="MovieID", columns="UserID", values="Rating")

SparseRate(rating_table)

90.396

In [18]:
rating = rating.join(table_film.set_index("MovieID"), on="MovieID")

rating = rating.drop("Timestamp", axis="columns")

rating.iloc[:5]

,UserID,MovieID,Rating,Title,Genre
0,1,1,4.0,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,1,3,4.0,Grumpier Old Men (1995),Comedy Romance
2,1,6,4.0,Heat (1995),Action Crime Thriller
3,1,47,5.0,Seven (a.k.a. Se7en) (1995),Mystery Thriller
4,1,50,5.0,"Usual Suspects, The (1995)",Crime Mystery Thriller


In [19]:
rating_table.iloc[:5, :8]

UserID,1,4,6,7,10,11,15,16
MovieID,,,,,,,,
1,4.0,NaN,NaN,4.5,NaN,NaN,2.5,NaN
2,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
3,4.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN
6,4.0,NaN,4.0,NaN,NaN,5.0,NaN,NaN


In [20]:
rating_table = rating_table.fillna(0)

rating_table.iloc[:5, :8]

UserID,1,4,6,7,10,11,15,16
MovieID,,,,,,,,
1,4.0,0.0,0.0,4.5,0.0,0.0,2.5,0.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
6,4.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0
